# xml

> This module has commands to handle the conversion of xml documents to line and box classes and vice versa

In [ ]:
#| default_exp manuscriptFiles

In [ ]:
#| hide
from nbdev.showdoc import *

In [8]:
#| export
from glyptodon.classes import *

## Primer on XML

In broadest terms, XML is not a markup language but a *standard*. XML is simply a set of rules for writing/marking up the contents of a document ([especially those stored on the internet](https://en.wikipedia.org/wiki/Markup_language#History_of_markup_languages)). It is a simple set of rules that also defines standards for extending the format and creating new rules. The fact that there are files with the ```.xml``` extension is reflective of these being files which adhere to the standard, not that they do not have additional features. For this reason, parsing the content of XML files programatically will depend on which particular XML standard is in use.

For more detail on this, read [this](http://archive1.village.virginia.edu/dvp4c/xmlintro.html) or [this](https://www.oreilly.com/library/view/xml-in-a/0596007647/ch01s05.html).

There are three major standards of XML currently used in digital humanities research on manuscripts. This statement is not the result of any complicated research on my end or any citation. It is just what appears to be the fact of the matter.

### [TEI](https://tei-c.org/about/history/)

The Text Encoding Initiative is the first of several groups that put together an XML standard specifically for the purposes of encoding published (for a certain understanding of published) textual material digitally. The encoding of a document was meant to make it possible to use digital tools to analyze texts. This was started in the US during the year of 1987, something that appears to this writer to coincide with the [rise of statistical methods of natural language processing in the 1990s](https://en.wikipedia.org/wiki/Natural_language_processing#Statistical_NLP_(1990s%E2%80%932010s)).

For the purposes of working on manuscripts, TEI XML standards are still important. Many research methods were built using TEI.

### [ALTO](https://cordis.europa.eu/project/id/IST-1999-20021)

The Analyzed Layout and Text Object schema was released in 2004. It exists alongside the [Metadata Encoding and Transmission Standard](https://en.wikipedia.org/wiki/Metadata_Encoding_and_Transmission_Standard) in order to describe a whole digitized document, not just one page. ALTO is now housed with the [Library of Congress](https://www.loc.gov/standards/alto/).

ALTO was designed for both [optical character recognition](https://en.wikipedia.org/wiki/Optical_character_recognition) and [document layout analysis](https://en.wikipedia.org/wiki/Document_layout_analysis). Notably, it is a default output of the [kraken](https://kraken.re/2.0.0/index.html) OCR system.

### [PAGE](https://github.com/PRImA-Research-Lab/PAGE-XML)

PAGE XML was released sometime before 2010. It was designed to be able to contain both *ground-truth* and the results of an OCR system so that the two could be directly compared. It is used as a default output format of both [Transkribus and eSriptorium](https://en.wikipedia.org/wiki/Page_Analysis_and_Ground_Truth_Elements)

There are other XML formats that are used to encode human written documents (to use the term loosely) and metadata, but these do not seem to be important formats for manuscripts:
- [RDF/XML](https://www.w3.org/TR/rdf11-primer/)
- [DC](https://www.dublincore.org/schemas/xmls/)
- [EAD](https://www.loc.gov/ead/)
- [ISO 12083](https://en.wikipedia.org/wiki/ISO_12083)
- [ABBYY XML](https://support.abbyy.com/hc/en-us/articles/360017336699-ABBYY-FineReader-Engine-XML-Export)

This list is non-exhaustive. As XML is an extensible standard that has existed for over two decades (first released in 1998), it has many variants and there may yet be more variants that will overtake the current three as a standard in the digital humanities.

## PAGE to Lines and BBoxes

We need to have a method which takes a file in the PAGE XML standard and allows us to instantiate Line and BBox from ```classes``` (with annotations) that are then written to a CSV. It should take in a file, a filename, and a directory and then call the various ```ToCSV``` methods on hand after manipulating the file for information.

In the [Stavronikita no.53](https://zenodo.org/record/5595669) dataset, a page is declared using:
```xml
<PcGts xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15">
```
All elements of a transcription are usually children of this element. If it is more convenient to separate the different pages into different files, one will usually just make each page such that is nested in the ```PcGts``` element and then declaring a new page with:
```xml
  <Page imageFilename="15_01_0053_0006_f_3r_res.png" imageWidth="1577" imageHeight="1995">
```
Beyond this though, a text line is captured in XML with:
```xml
      <TextLine id="r0l0">
        <Coords points="349,663 1135,663 1135,731 349,731" />
```
The white space at the beginning is important in order to make sure that the ```Coords``` are accessible. All words are children of the ```TextLine``` object such that:
```xml
        <Word id="r0l0w2">
          <Coords points="349,669 534,669 534,724 349,724" />
```
This is quite similar to the ```TextLine``` object. After all the ```Word``` objects are enumerated, the text on the line is specified by:
```xml
        <TextEquiv>
          <Unicode>Βίβλος γενέσεως ἰῦ χῦ· υἱοῦ δαδ·</Unicode>
        </TextEquiv>
      </TextLine>
```
Similarly, text is assigned to words with:
```xml
          <TextEquiv>
            <Unicode>γενέσεως</Unicode>
          </TextEquiv>
        </Word>
```
By the [PAGE schema](https://www.primaresearch.org/schema/PAGE/gts/pagecontent/2013-07-15/pagecontent.xsd) used, nested in a ```TextLine``` object can be the following categories in this precise order:
1. Coords
2. Baseline
3. Word
4. TextEquiv
5. TextStyle

Therefore, immediately after a ```TextLine``` is defined, the coordinates are defined. However, the ```TextEquiv``` is assigned after all the ```Word``` objects are assigned. This means that the best way to build the classes out is to nest the ```Word``` level declarations (i.e. words to bboxes) inside the ```TextLine``` level declarations.

There are other important considerations (such as ```TextRegion``` objects corresponding to columns), but for a single column manuscript the above considerations are sufficient.

## ALTO to Lines and BBoxes

We need to have a method which takes a file in the ALTO XML standard and allows us to instantiate Line and BBox from ```classes``` (with annotations) that are then written to a CSV. It should take in a file, a filename, and a directory and then call the various ```ToCSV``` methods on hand after manipulating the file for information.

In the [CREMMA Medieval](https://github.com/HTR-United/cremma-medieval) dataset, the ALTO schema is used ([this one](https://gitlab.inria.fr/scripta/escriptorium/-/raw/develop/app/escriptorium/static/alto-4-1-baselines.xsd) in particular).

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()